# Semantic Similarity - Experiment 01
The objective of this trial is to expand the SCA_index (i.e., Semantic Content Analysis Index) to a full word embedding, setting a subjective or objective load for each word.

## Introduction

### Libraries

In [239]:
## Data analysis packages:
import pandas as pd
import numpy as np
from math import isnan  #Verifies if a given value is numerical.
import re  # Regular Expression

In [240]:
## Visualization packages:
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Definitions

In [241]:
## Forcing Pandas to display any number of elements
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)

## SpaCy Word Embeddings: 
Also using Spacy library: https://spacy.io/
> !pip install -U spacy  
> !python -m spacy download en_core_web_sm  
> !python -m spacy download en_core_web_lg

Some instructions on how to use it:  
https://spacy.io/usage/spacy-101

In [242]:
## Importing SpaCy library:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg")

In [243]:
## We check below that this model has 514.157 keys and vectors, respectively.
nlp.meta['vectors']

{'width': 300,
 'vectors': 514157,
 'keys': 514157,
 'name': 'en_vectors',
 'mode': 'default'}

In [244]:
## Again, checking the number of keys.
nlp.vocab.vectors.n_keys

514157

In [323]:
## Finding the SpaCy index for a given word:
nlp.vocab.strings['any']

13148361048351484388

In [324]:
## Getting the text and vector for a given index:
tmp_Idx = 13148361048351484388

tmp_vector = nlp.vocab[tmp_Idx].vector
tmp_text = nlp.vocab[tmp_Idx].text

In [325]:
print(f'{tmp_Idx}: {tmp_text}:\n{tmp_vector[0:3]}...')

13148361048351484388: any:
[-0.96335  3.714   -2.3964 ]...


In [249]:
## Defining a method to get vector and text information for a word:
def nlp_getVector(word, verbose=False):
    '''
    Obtains the vector representation of a given word from SpaCy word embedding.
    Usage:  nlp_getVector(word)[0] to get the text; nlp_getVector(word)[1] to get the vector.
            var_text, var_vector = nlp_getVector(word)
    '''
    ## Generates the word hash:
    hash = nlp.vocab.strings[word]
    try:
        word_vector = nlp.vocab[hash].vector
        word_text = nlp.vocab[hash].text
    except:
        if verbose:
            print('Error: word vector not available.')
        return None
    return (word_text, word_vector)

In [259]:
## Testing the method:
print(f'Word: {nlp_getVector("study")[0]} <-> Vector 3rd elements: {nlp_getVector("study")[1][:3]}')

Word: study <-> Vector 3rd elements: [-0.64776 -1.3091   2.0104 ]


---
## SCA - Glasgow Norms
* Read the SCA from Glasgow Norms;  
* Import F_s and F_o from the previous study;  
* Train the MLP classifier.

In [11]:
df_factors = pd.read_csv('../data/df_factors.csv', sep=';')
df_factors.head()

,words,F_Objectivity,F_Subjectivity,F_Context
0,abattoir,0.512527,0.380603,0.960466
1,abbey,0.714765,0.240456,0.696198
2,abbreviate,0.286952,0.171052,0.767043
3,abdicate,0.144736,0.384300,0.863127
4,abdication,0.167654,0.334086,0.896733


### DataPrep: De-Duplicating words  
There are words in the Glasgow Norms that were differentiated from their homonymous, such as 'case'. In this section, we first select those words and then input a mean value for them.

In [266]:
## Exemplifying the need for de-duplication:
df_factors[df_factors['words'].str.contains('case')]

,words,F_Objectivity,F_Subjectivity,F_Context
533,bookcase,0.926393,0.374441,0.335542
756,case,0.715863,0.164100,0.409611
757,case (container),0.821136,0.079956,0.400335
758,case (instance),0.233820,0.213528,0.651273
759,case (legal),0.456260,0.369828,0.733642
4808,suitcase,0.961068,0.256584,0.356338


In [13]:
## Creating a new dataframe by splitting the 'words' column into two columns:
df_homonym = df_factors.copy()
df_homonym[['word','distinction']] = df_homonym['words'].str.split('(', expand=True)
## Renaming the columns of the new dataframe:
# df_homonym.columns = ['word','distinction']

# Stripping whitespace from the new columns
df_homonym['word'] = df_homonym['word'].str.strip()
df_homonym['distinction'] = df_homonym['distinction'].str.strip().str.rstrip(')')

## Showing dataframe:
df_homonym.head(5)

,words,F_Objectivity,F_Subjectivity,F_Context,word,distinction
0,abattoir,0.512527,0.380603,0.960466,abattoir,None
1,abbey,0.714765,0.240456,0.696198,abbey,None
2,abbreviate,0.286952,0.171052,0.767043,abbreviate,None
3,abdicate,0.144736,0.384300,0.863127,abdicate,None
4,abdication,0.167654,0.334086,0.896733,abdication,None


In [15]:
# Calculating the number of distinct elements in the 'distinction' column for each word in the 'word' column
distinct_count = df_homonym.groupby('word')['distinction'].nunique().reset_index()

# Counting the frequency of different numbers of distinct elements
pivot_table = distinct_count.groupby('distinction')['word'].count().reset_index()

# Renaming the columns
pivot_table.columns = ['n_distinction', 'n_words']
pivot_table

,n_distinction,n_words
0,0,4303
1,1,2
2,2,288
3,3,69
4,4,19
5,5,2


#### Analyzing the words with 5 distinct meaning:

In [18]:
## Getting the word with the highest number of distinct meanings: 
distinct_count[distinct_count['distinction'] == 5]

,word,distinction
721,charge,5
822,club,5


In [22]:
df_homonym[(df_homonym['word']=='club') | (df_homonym['word']=='charge')]

,words,F_Objectivity,F_Subjectivity,F_Context,word,distinction
819,charge,0.340881,0.330534,0.438418,charge,None
820,charge (card),0.477083,0.293352,0.671798,charge,card
821,charge (dependent),0.256759,0.164572,0.791042,charge,dependent
822,charge (electric),0.391504,0.413080,0.697287,charge,electric
823,charge (price),0.428509,0.353942,0.498770,charge,price
824,charge (rush),0.551191,0.557549,0.591914,charge,rush
942,club,0.774815,0.476734,0.467275,club,None
943,club (card suit),0.773094,0.187918,0.492067,club,card suit
944,club (disco),0.863845,0.547812,0.658553,club,disco
945,club (hit),0.725571,0.435103,0.607930,club,hit


#### Replacing the distinct values for the average (column 'word'):

In [24]:
# Calculating the mean of F_Objectivity and F_Subjectivity for each group of "word"
mean_values = df_homonym.groupby('word')[['F_Objectivity', 'F_Subjectivity']].mean().reset_index()

In [25]:
mean_values

,word,F_Objectivity,F_Subjectivity
0,Apple,0.940620,0.524376
1,Christmas,0.850793,0.833898
2,Dad,0.856533,0.493834
3,Dame,0.626968,0.300580
4,FALSE,0.156905,0.473624
...,...,...,...
4678,zeppelin,0.864760,0.396531
4679,zero,0.379392,0.315118
4680,zest,0.402894,0.476800
4681,zoo,0.867152,0.507390


In [111]:
## Merging the mean values back into the original DataFrame
df = pd.merge(df_homonym, mean_values, on='word', suffixes=('', '_mean'))

## Checking an example (is the "_mean" values equal for all instances?): 
df[df['word'] == 'charge']

,words,F_Objectivity,F_Subjectivity,F_Context,word,distinction,F_Objectivity_mean,F_Subjectivity_mean
819,charge,0.340881,0.330534,0.438418,charge,None,0.407655,0.352172
820,charge (card),0.477083,0.293352,0.671798,charge,card,0.407655,0.352172
821,charge (dependent),0.256759,0.164572,0.791042,charge,dependent,0.407655,0.352172
822,charge (electric),0.391504,0.413080,0.697287,charge,electric,0.407655,0.352172
823,charge (price),0.428509,0.353942,0.498770,charge,price,0.407655,0.352172
824,charge (rush),0.551191,0.557549,0.591914,charge,rush,0.407655,0.352172


In [112]:
## Dropping duplicated words (by column)
df.drop_duplicates(subset=['word'], inplace=True)

## Selecting only the columns of interest:
df = df[['word', 'F_Objectivity_mean', 'F_Subjectivity_mean']]

## Renaming the columns to remove the "_mean" suffix
df.rename(columns={'F_Objectivity_mean': 'F_Objectivity', 'F_Subjectivity_mean': 'F_Subjectivity'}, inplace=True)

In [113]:
## Showing dataframe:
print(df.info())
df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4683 entries, 0 to 5552
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   word            4683 non-null   object 
 1   F_Objectivity   4683 non-null   float64
 2   F_Subjectivity  4683 non-null   float64
dtypes: float64(2), object(1)
memory usage: 146.3+ KB
None


,word,F_Objectivity,F_Subjectivity
0,abattoir,0.512527,0.380603
1,abbey,0.714765,0.240456
2,abbreviate,0.286952,0.171052
3,abdicate,0.144736,0.384300
4,abdication,0.167654,0.334086


In [114]:
## Saving the prepared data:
df.to_csv('../data/df_factors_prepared.csv')

In [115]:
# ### Selecionando apenas as palavras no df_factors que atendam aos critérios:
# df_selected = df.loc[((df['F_Subjectivity'] > 0.75) | (df['F_Subjectivity'] < 0.3)) & ((df['F_Objectivity'] > 0.75) | (df['F_Objectivity'] < 0.3))]
# df_factors = df_selected.copy()

In [116]:
## Generating a list of words from SCA for training.
SCA_words = [word for word in df.word]
len(SCA_words)

4683

In [117]:
## Preparing dataframe for training:
# df.set_index('word', inplace=True)

In [118]:
df

,word,F_Objectivity,F_Subjectivity
0,abattoir,0.512527,0.380603
1,abbey,0.714765,0.240456
2,abbreviate,0.286952,0.171052
3,abdicate,0.144736,0.384300
4,abdication,0.167654,0.334086
...,...,...,...
5548,zeppelin,0.864760,0.396531
5549,zero,0.379392,0.315118
5550,zest,0.402894,0.476800
5551,zoo,0.867152,0.507390


### Splitting SCA-Glasgow Norms data into train and test:

In [119]:
from sklearn.model_selection import train_test_split

# Splitting data in train (75%) and test (25%)
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [120]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3512 entries, 213 to 1012
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   word            3512 non-null   object 
 1   F_Objectivity   3512 non-null   float64
 2   F_Subjectivity  3512 non-null   float64
dtypes: float64(2), object(1)
memory usage: 109.8+ KB


In [272]:
def generateData(df_sca:pd.DataFrame):
    '''Creates a dataset with explanatory variables from word embedding and dependent variables based on the SCA target data.
    Usage example: "X_train, Y_train = generateData(df)"
    Input: Pandas.DataFrame with SCA words.
    Output: X: dataframe with explanatory variables; Y: dataframe with dependent variables.
    '''
    X = {}
    Y = {}
    
    for i, row in df_sca.iterrows():
        word = row['word']
        f_objectivity = row['F_Objectivity']
        f_subjectivity = row['F_Subjectivity']

        try:
            X[word] = nlp_getVector(word)[1]  # Stores the word vector
            Y[word] = {'F_Objectivity': f_objectivity, 'F_Subjectivity': f_subjectivity}
            # print(f'Debug: {word} <=> {nlp_getVector(word)[0]}')
        except:
            pass

    return pd.DataFrame.from_dict(X, orient='index'), pd.DataFrame.from_dict(Y, orient='index')

In [273]:
## Generating data:
X_train, Y_train = generateData(train_df)
X_test, Y_test = generateData(test_df)

In [274]:
## Checking the generated data dimension:
print("Train data dimension:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("\nTest data dimension:")
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)

Train data dimension:
X_train: (3511, 300)
Y_train: (3511, 2)

Test data dimension:
X_test: (1170, 300)
Y_test: (1170, 2)


In [326]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
argument,-2.39920,-0.31061,0.162480,1.19390,4.55300,2.608000,2.381600,4.37930,-6.4059,-2.898900,6.9065,3.635700,-3.4818,-3.07200,-1.562500,4.45740,2.406200,0.61288,-2.02030,1.85400,-1.28270,-0.866920,-2.234400,-0.202930,-0.98764,-2.57030,-0.42565,-0.67371,-2.95120,2.52310,3.61740,2.046000,0.93659,0.80884,-1.835900,-4.15380,-0.77178,-0.91474,0.357400,2.43110,-2.84270,-4.39410,0.18871,1.04750,-0.79925,-1.58430,3.480400,-2.895800,-1.462500,0.56112,-2.86750,2.770100,2.391500,-1.90630,-2.25350,-0.62433,3.14550,1.222000,2.28540,0.36479,3.091800,1.61080,-1.65840,-1.64510,0.65175,2.85370,-3.29310,-4.5797,1.7337,3.35200,1.768100,1.599000,-5.05160,1.664400,-4.369200,1.14050,-3.99870,5.094700,-0.79874,-0.608100,-3.21580,-0.42495,1.404700,2.54230,0.58327,2.52930,-3.55850,-2.18920,4.41760,1.643900,3.472500,2.23200,5.642900,-2.55720,3.22990,-2.32170,0.044789,3.106200,2.82930,-2.23330,3.69810,3.2175,4.477000,3.126400,-0.69679,3.03720,3.25990,-4.15510,1.04660,-2.30530,0.066955,-0.960050,-0.48711,1.059800,1.91210,2.15400,-4.61480,-0.606290,0.35795,-0.86739,-0.40956,-5.43570,0.59216,-1.75330,-3.36790,0.397490,3.54500,-3.55280,0.711020,-4.57000,-4.58250,-1.846200,3.95070,0.42573,0.61037,0.72052,0.78121,2.548200,0.79837,-1.85340,-1.46980,-0.71325,-0.21927,1.68080,1.2657,-0.95549,-4.65340,-0.422500,-0.655770,3.78360,0.58572,4.75820,-1.281200,1.68730,-1.43430,5.10480,1.84110,-0.33209,-2.84530,-4.33760,2.24460,0.060355,-0.28312,2.110900,-0.214760,-2.08980,-2.99090,1.68460,1.22150,-1.663000,-0.29140,-0.15043,-0.089668,-1.361800,4.252300,1.049900,-0.835560,3.34520,-2.71990,-3.49400,2.819600,3.07220,-0.129980,1.63730,-1.97820,2.92380,-2.132400,-1.18320,3.44950,2.63860,-2.62280,-3.05720,0.773540,1.61730,2.12610,-3.067000,-0.84982,3.68300,1.61980,6.20910,-2.488200,0.70562,0.360680,-0.85727,2.07030,-0.057408,-0.638360,0.628880,-0.26941,-1.863700,-2.42160,-1.494200,-2.671000,4.4052,-0.897830,1.84780,1.427400,1.16620,4.00380,2.206200,-3.77060,-0.94775,0.081936,-0.052859,-1.14660,0.97386,1.65780,5.16590,1.97350,-0.10786,1.467100,-4.17920,1.528100,0.131100,1.19560,0.493110,2.88420,1.03060,2.35040,-2.68750,-1.00970,1.24780,1.294900,4.564800,0.517240,0.524770,-2.016200,2.089300,0.606990,1.592600,1.05390,-4.59500,-3.70100,-0.75894,-0.64485,-4.18290,-1.93710,-0.26863,0.26782,-0.63142,-2.10990,1.153400,1.66280,4.74830,0.868380,0.88791,4.310000,4.37190,-3.07340,-1.11620,-0.311190,-1.498500,-1.209900,0.359000,3.47040,-0.40592,0.012269,-1.62150,-1.38370,4.22990,1.46310,0.48673,1.717800,3.760400,4.84590,1.29130,4.0310,-0.377380,-3.63400,0.84509,3.23500,-1.73110,1.399600,2.05100,-4.25220,1.091100,3.78190,-0.92356,-1.8916,3.350400
awful,-0.33203,-0.35220,-0.011708,-0.88964,-0.75291,1.845700,0.623790,1.01190,2.1302,1.779500,1.2335,-1.207700,-1.6876,0.33506,0.376710,0.83856,0.031083,-4.43120,-0.83196,-1.03640,-1.26410,1.848000,-0.288830,-1.527500,0.73359,-1.55750,0.28682,-0.65072,-0.79270,1.72920,0.324

In [275]:
Y_train

,F_Objectivity,F_Subjectivity
argument,0.365722,0.647198
awful,0.201122,0.507171
decorator,0.774472,0.291107
misconception,0.013464,0.474901
crab,0.816438,0.227631
...,...,...
vaccinate,0.678125,0.443751
boastful,0.437815,0.575248
phrase,0.265757,0.144886
ski,0.900524,0.370531


---
### Training a MLP Classifier for word semantic content

In [134]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [346]:
### Model 01: MLP architecture for continuous output:
model = Sequential([
    Dense(256, activation='relu', input_shape=(300,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    # Dense(2, activation='linear')  # 2 neurons for continuous output between 0 and 1
    Dense(2, activation='sigmoid')  # 2 neurons for continuous output between 0 and 1
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [347]:
# Print model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 256)               77056     
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 32)                2080      
                                                                 
 dense_24 (Dense)            (None, 2)                 66        
                                                                 
Total params: 120354 (470.13 KB)
Trainable params: 120354 (470.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [348]:
## Transforming data into numpy array:
Y_train_array = Y_train.to_numpy()
Y_test_array = Y_test.to_numpy()

X_train_array = X_train.to_numpy()
X_test_array = X_test.to_numpy()

# Print the first few elements to verify
print(Y_train_array[:5])  # Print the first 5 elements


[[0.36572243 0.64719766]
 [0.20112169 0.50717064]
 [0.77447234 0.29110657]
 [0.01346386 0.47490132]
 [0.81643835 0.2276313 ]]


In [349]:
# Train the model
history = model.fit(X_train_array, Y_train_array, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
176/176 [==============================] - 1s 3ms/step - loss: 0.0263 - accuracy: 0.8187 - val_loss: 0.0201 - val_accuracy: 0.8578
Epoch 2/50
176/176 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.8864 - val_loss: 0.0171 - val_accuracy: 0.8563
Epoch 3/50
176/176 [==============================] - 0s 2ms/step - loss: 0.0103 - accuracy: 0.8967 - val_loss: 0.0157 - val_accuracy: 0.8649
Epoch 4/50
176/176 [==============================] - 0s 2ms/step - loss: 0.0080 - accuracy: 0.9177 - val_loss: 0.0167 - val_accuracy: 0.8620
Epoch 5/50
176/176 [==============================] - 0s 2ms/step - loss: 0.0063 - accuracy: 0.9266 - val_loss: 0.0163 - val_accuracy: 0.8620
Epoch 6/50
176/176 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 0.9430 - val_loss: 0.0163 - val_accuracy: 0.8706
Epoch 7/50
176/176 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 0.9348 - val_loss: 0.0164 - val_accuracy: 0.8691
Epoch 

In [350]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_array, Y_test_array)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

37/37 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 0.8752
Test Accuracy: 87.52%


### Applying the trained model to predict SCA for a distinct word:

In [391]:
## Choosing an arbitrary word:
entry = 'monster'

In [392]:
## Getting its vector representation (from word embedding) and preparing it to the model input format:
new_entry = nlp_getVector(entry)[1]
new_entry = np.expand_dims(new_entry, axis=0)

In [393]:
## Showing how this entry is laid out:
new_entry

array([[ 1.1305  , -1.5563  , -2.4944  , -2.024   , -1.5499  ,  2.395   ,
         0.91659 ,  1.5593  , -0.32409 ,  3.3341  ,  2.9497  , -1.1545  ,
         0.41695 ,  4.3341  ,  2.2508  , -0.35554 ,  4.7371  , -1.1894  ,
         1.1579  ,  0.65003 ,  1.4585  ,  2.6705  , -0.27433 , -3.0269  ,
         1.3221  ,  0.39813 , -2.5172  , -3.6463  ,  1.5057  ,  4.1713  ,
        -1.1484  , -0.57279 ,  3.2361  , -0.98623 , -1.0243  , -1.5263  ,
         1.067   ,  1.5488  ,  2.2951  ,  1.5047  , -0.72364 , -0.62793 ,
         1.0891  ,  2.0448  , -0.78437 ,  0.063764, -2.2023  , -2.2536  ,
         0.92767 ,  2.184   ,  0.35328 ,  2.0611  ,  2.7751  , -0.53082 ,
        -0.069744,  0.10063 ,  0.75115 ,  0.10459 ,  0.75907 ,  2.3196  ,
         1.7684  , -0.10156 , -2.3969  , -0.51817 ,  0.83881 ,  1.4263  ,
        -4.3255  , -3.3691  ,  0.64677 , -1.3543  ,  0.4567  ,  2.7448  ,
         0.10841 ,  0.99697 , -0.8093  , -1.0998  , -0.25594 ,  2.2797  ,
         1.1398  , -0.62884 , -2.5255 

In [394]:
## Predicting the objective and subjective loads for the 'entry' word:
result = model.predict(new_entry)
result

1/1 [==============================] - 0s 39ms/step


array([[0.7303195 , 0.74615556]], dtype=float32)

In [395]:
## Generalizing the process above for any word through a python method:
def wordClassifier(word):
    '''Given a word vector, shows the probability for objective and subjective semantic content, respectively.'''
    new_entry = nlp_getVector(word)
    if new_entry:
        vector = nlp_getVector(word)[1]
        vector = np.expand_dims(vector, axis=0)
        result = model.predict(vector)
    else:
        print('Word not existent in database.')
        return
    print(f'--- {word}:\n{result[0][0]*100:.2f} of objectivity\n{result[0][1]*100:.2f} of subjectivity')
    return


In [397]:
trial_word = 'hippopotamus'
wordClassifier(trial_word)

1/1 [==============================] - 0s 26ms/step
--- hippopotamus:
95.31 of objectivity
38.44 of subjectivity


In [398]:
df_factors[df_factors['words']==trial_word]

,words,F_Objectivity,F_Subjectivity,F_Context
2387,hippopotamus,0.973593,0.377959,0.309328


----
### Training a model for multilabel classification:

In [ ]:
# Define a new MLP architecture
model = Sequential([
    Dense(256, activation='relu', input_shape=(300,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # 2 neurons for binary classification with softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use categorical_crossentropy for categorical data
              metrics=['accuracy'])

In [ ]:
# Como converti a saída em dados categóricos, é preciso antes utilizar OneHotEncoder:
# Use categorical_crossentropy for categorical data

#### Binarizing Y_train and Y_test
Once we run the first MLP model, the performance wasn't over 54%. 
In this Section, we will binarize the semantic factor values following the median values.

In [ ]:
# Binarizing through list comprehension
Y_train['F_Objectivity'] = ['high' if f_objectivity >= 0.565 else 'low' for f_objectivity in Y_train['F_Objectivity']]
Y_train['F_Subjectivity'] = ['high' if f_subjectivity >= 0.392 else 'low' for f_subjectivity in Y_train['F_Subjectivity']]

Y_test['F_Objectivity'] = ['high' if f_objectivity >= 0.565 else 'low' for f_objectivity in Y_test['F_Objectivity']]
Y_test['F_Subjectivity'] = ['high' if f_subjectivity >= 0.392 else 'low' for f_subjectivity in Y_test['F_Subjectivity']]

In [ ]:
# Binarizing through list comprehension
Y_train['F_Objectivity'] = [1 if f_objectivity >= 0.565 else 0 for f_objectivity in Y_train['F_Objectivity']]
Y_train['F_Subjectivity'] = [1 if f_subjectivity >= 0.392 else 0 for f_subjectivity in Y_train['F_Subjectivity']]

Y_test['F_Objectivity'] = [1 if f_objectivity >= 0.565 else 0 for f_objectivity in Y_test['F_Objectivity']]
Y_test['F_Subjectivity'] = [1 if f_subjectivity >= 0.392 else 0 for f_subjectivity in Y_test['F_Subjectivity']]

In [ ]:
# Convert multilabel categorical labels to binary vectors
multi_label_binarizer = MultiLabelBinarizer()
Y_train_encoded = multi_label_binarizer.fit_transform(Y_train)
Y_test_encoded = multi_label_binarizer.transform(Y_test)

In [ ]:
len(Y_train_encoded)

2

In [ ]:
Y_train_encoded

array([[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

---
### Choosing outperformed model through PyCaret:

In [322]:
import pycaret.classification
import pycaret.regression

> Adjustin Y_train for use with PyCaret:

In [148]:
df = X_train.copy()

In [149]:
# Definir uma função para mapear os valores das colunas para os rótulos desejados
def map_labels(row):
    if row['F_Objectivity'] == 0 and row['F_Subjectivity'] == 1:
        return 'Latent'
    elif row['F_Objectivity'] == 0 and row['F_Subjectivity'] == 0:
        return 'Contextual'
    elif row['F_Objectivity'] == 1 and row['F_Subjectivity'] == 0:
        return 'Manifest'
    elif row['F_Objectivity'] == 1 and row['F_Subjectivity'] == 1:
        return 'Perceptual'

In [154]:
# Aplicar a função de mapeamento para criar a nova coluna "Target"
df['target'] = Y_train.apply(map_labels, axis=1)

In [151]:
## Adding F_Subjectivity and F_Objectivity to df:
df['F_Subjectivity'] = Y_train['F_Subjectivity']
df['F_Objectivity'] = Y_train['F_Objectivity']

In [152]:
df['target'] = Y_train['F_Subjectivity']

In [155]:
# Exibir as primeiras linhas do DataFrame resultante para verificar
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,target,F_Subjectivity,F_Objectivity
argument,-2.39920,-0.31061,0.162480,1.19390,4.55300,2.60800,2.381600,4.3793,-6.40590,-2.89890,6.90650,3.63570,-3.48180,-3.07200,-1.56250,4.45740,2.406200,0.61288,-2.02030,1.854000,-1.28270,-0.866920,-2.23440,-0.20293,-0.98764,-2.570300,-0.42565,-0.67371,-2.95120,2.52310,3.617400,2.046000,0.93659,0.80884,-1.83590,-4.15380,-0.77178,-0.91474,0.357400,2.43110,-2.84270,-4.39410,0.18871,1.04750,-0.79925,-1.584300,3.480400,-2.895800,-1.462500,0.56112,-2.86750,2.77010,2.39150,-1.90630,-2.25350,-0.62433,3.14550,1.22200,2.28540,0.36479,3.091800,1.61080,-1.658400,-1.64510,0.651750,2.85370,-3.29310,-4.579700,1.73370,3.35200,1.76810,1.59900,-5.05160,1.664400,-4.36920,1.14050,-3.99870,5.09470,-0.798740,-0.608100,-3.21580,-0.42495,1.40470,2.54230,0.58327,2.529300,-3.55850,-2.189200,4.41760,1.6439,3.47250,2.23200,5.642900,-2.557200,3.22990,-2.32170,0.044789,3.10620,2.82930,-2.233300,3.6981,3.217500,4.477000,3.12640,-0.69679,3.03720,3.259900,-4.15510,1.046600,-2.30530,0.066955,-0.960050,-0.487110,1.05980,1.91210,2.15400,-4.61480,-0.60629,0.35795,-0.867390,-0.40956,-5.43570,0.59216,-1.75330,-3.36790,0.397490,3.545000,-3.552800,0.71102,-4.57000,-4.5825,-1.846200,3.95070,0.42573,0.61037,0.720520,0.78121,2.54820,0.79837,-1.85340,-1.46980,-0.713250,-0.219270,1.68080,1.26570,-0.955490,-4.65340,-0.422500,-0.655770,3.783600,0.58572,4.75820,-1.28120,1.68730,-1.43430,5.10480,1.841100,-0.33209,-2.84530,-4.337600,2.24460,0.060355,-0.28312,2.110900,-0.21476,-2.089800,-2.99090,1.68460,1.2215,-1.663000,-0.291400,-0.15043,-0.089668,-1.36180,4.25230,1.04990,-0.835560,3.345200,-2.71990,-3.49400,2.81960,3.07220,-0.12998,1.63730,-1.97820,2.92380,-2.13240,-1.18320,3.449500,2.63860,-2.62280,-3.05720,0.77354,1.61730,2.12610,-3.06700,-0.84982,3.68300,1.61980,6.20910,-2.488200,0.70562,0.360680,-0.85727,2.07030,-0.057408,-0.638360,0.62888,-0.26941,-1.86370,-2.42160,-1.494200,-2.671000,4.4052,-0.897830,1.847800,1.427400,1.16620,4.00380,2.20620,-3.770600,-0.94775,0.081936,-0.052859,-1.14660,0.97386,1.65780,5.16590,1.97350,-0.10786,1.46710,-4.17920,1.52810,0.131100,1.195600,0.493110,2.88420,1.03060,2.35040,-2.687500,-1.009700,1.2478,1.29490,4.56480,0.51724,0.524770,-2.016200,2.089300,0.60699,1.59260,1.05390,-4.595000,-3.70100,-0.75894,-0.64485,-4.18290,-1.93710,-0.26863,0.26782,-0.63142,-2.10990,1.153400,1.66280,4.74830,0.868380,0.88791,4.310000,4.37190,-3.073400,-1.11620,-0.311190,-1.498500,-1.20990,0.35900,3.47040,-0.40592,0.012269,-1.62150,-1.3837,4.22990,1.46310,0.48673,1.71780,3.760400,4.84590,1.29130,4.0310,-0.37738,-3.63400,0.84509,3.23500,-1.73110,1.39960,2.05100,-4.25220,1.091100,3.78190,-0.92356,-1.8916,3.35040,Latent,1,0
awful,-0.33203,-0.35220,-0.011708,-0.88964,-0.75291,1.84570,0.623790,1.0119,2.13020,1.77950,1.23350,-1.20770,-1.68760,0.33506,0.37671,0.83856,0.031083,-4.43120,-0.83196,-1.036400,-1.26410,1.848000,-0.28883,-1.52750,0.73359,-1.557500,0.2868

In [156]:
## Descartando a última coluna:
df_last = df.iloc[:,:-2]
df_last.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,target
argument,-2.3992,-0.31061,0.16248,1.1939,4.553,2.608,2.3816,4.3793,-6.4059,-2.8989,6.9065,3.6357,-3.4818,-3.072,-1.5625,4.4574,2.4062,0.61288,-2.0203,1.854,-1.2827,-0.86692,-2.2344,-0.20293,-0.98764,-2.5703,-0.42565,-0.67371,-2.9512,2.5231,3.6174,2.046,0.93659,0.80884,-1.8359,-4.1538,-0.77178,-0.91474,0.3574,2.4311,-2.8427,-4.3941,0.18871,1.0475,-0.79925,-1.5843,3.4804,-2.8958,-1.4625,0.56112,-2.8675,2.7701,2.3915,-1.9063,-2.2535,-0.62433,3.1455,1.222,2.2854,0.36479,3.0918,1.6108,-1.6584,-1.6451,0.65175,2.8537,-3.2931,-4.5797,1.7337,3.352,1.7681,1.599,-5.0516,1.6644,-4.3692,1.1405,-3.9987,5.0947,-0.79874,-0.6081,-3.2158,-0.42495,1.4047,2.5423,0.58327,2.5293,-3.5585,-2.1892,4.4176,1.6439,3.4725,2.232,5.6429,-2.5572,3.2299,-2.3217,0.044789,3.1062,2.8293,-2.2333,3.6981,3.2175,4.477,3.1264,-0.69679,3.0372,3.2599,-4.1551,1.0466,-2.3053,0.066955,-0.96005,-0.48711,1.0598,1.9121,2.154,-4.6148,-0.60629,0.35795,-0.86739,-0.40956,-5.4357,0.59216,-1.7533,-3.3679,0.39749,3.545,-3.5528,0.71102,-4.57,-4.5825,-1.8462,3.9507,0.42573,0.61037,0.72052,0.78121,2.5482,0.79837,-1.8534,-1.4698,-0.71325,-0.21927,1.6808,1.2657,-0.95549,-4.6534,-0.4225,-0.65577,3.7836,0.58572,4.7582,-1.2812,1.6873,-1.4343,5.1048,1.8411,-0.33209,-2.8453,-4.3376,2.2446,0.060355,-0.28312,2.1109,-0.21476,-2.0898,-2.9909,1.6846,1.2215,-1.663,-0.2914,-0.15043,-0.089668,-1.3618,4.2523,1.0499,-0.83556,3.3452,-2.7199,-3.494,2.8196,3.0722,-0.12998,1.6373,-1.9782,2.9238,-2.1324,-1.1832,3.4495,2.6386,-2.6228,-3.0572,0.77354,1.6173,2.1261,-3.067,-0.84982,3.683,1.6198,6.2091,-2.4882,0.70562,0.36068,-0.85727,2.0703,-0.057408,-0.63836,0.62888,-0.26941,-1.8637,-2.4216,-1.4942,-2.671,4.4052,-0.89783,1.8478,1.4274,1.1662,4.0038,2.2062,-3.7706,-0.94775,0.081936,-0.052859,-1.1466,0.97386,1.6578,5.1659,1.9735,-0.10786,1.4671,-4.1792,1.5281,0.1311,1.1956,0.49311,2.8842,1.0306,2.3504,-2.6875,-1.0097,1.2478,1.2949,4.5648,0.51724,0.52477,-2.0162,2.0893,0.60699,1.5926,1.0539,-4.595,-3.701,-0.75894,-0.64485,-4.1829,-1.9371,-0.26863,0.26782,-0.63142,-2.1099,1.1534,1.6628,4.7483,0.86838,0.88791,4.31,4.3719,-3.0734,-1.1162,-0.31119,-1.4985,-1.2099,0.359,3.4704,-0.40592,0.012269,-1.6215,-1.3837,4.2299,1.4631,0.48673,1.7178,3.7604,4.8459,1.2913,4.031,-0.37738,-3.634,0.84509,3.235,-1.7311,1.3996,2.051,-4.2522,1.0911,3.7819,-0.92356,-1.8916,3.3504,Latent


In [75]:
df_scnd_last = df.iloc[:, :-2].join(df.iloc[:, -1])
df_scnd_last.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,target,F_Objectivity
periodical,-1.6272,-0.78238,1.6661,0.64415,5.8025,-3.8606,2.1145,2.2464,0.11389,-0.57345,5.7366,2.6897,-2.9695,2.0592,1.1753,2.1266,4.7035,2.7336,-3.5872,1.1187,-0.33484,-1.4071,-4.7561,2.2302,-2.059,-0.14411,-1.9042,-3.213,-0.34878,1.1597,-1.3406,0.47455,0.76463,0.62292,-2.3959,-0.086495,1.1585,-0.68091,1.1494,1.3205,1.8991,-0.035937,-0.36217,2.4583,1.1064,2.3078,-0.64062,0.28873,-1.6005,-2.3947,-2.2164,3.735,-0.78451,-0.50572,-2.4039,1.9877,-2.2064,2.1254,2.0112,-3.4274,0.38636,-0.99008,-2.2497,1.6211,3.0944,0.64471,1.4321,-0.62388,0.8677,1.1196,-1.9251,-2.9001,-2.8303,-2.8833,-0.33352,3.1787,-1.475,1.1811,-1.86,0.12378,-1.6496,-0.14419,-0.56384,1.1399,2.2299,3.6695,-1.0809,-2.7011,2.3617,-2.0884,-0.71105,1.6108,3.7237,-1.2607,2.374,-1.998,1.72,1.4512,-0.39968,4.1796,2.6991,2.5275,2.0868,1.2196,-1.0239,3.438,3.123,-3.8607,1.4855,-4.1848,3.7903,0.88335,-2.853,-0.98828,1.9463,1.1769,-1.8306,2.754,-1.7812,-1.4768,0.84724,-4.4108,1.7444,1.4066,1.8559,-4.2074,-0.42883,-2.2853,2.6108,-2.4452,-1.5658,0.6037,2.0717,2.7407,0.50984,0.82838,-1.9149,-1.2288,0.70129,-3.165,-0.40373,-1.5314,-0.56426,1.6354,3.8474,-0.94437,-0.99321,-0.052637,0.75979,3.275,2.3954,0.05155,0.77701,1.3314,-1.7912,0.54084,-0.39345,-0.94512,-1.1933,-1.5246,-1.9794,1.4443,1.1399,3.4458,-2.8027,-1.2135,-6.3629,1.6438,-0.24045,0.16754,1.4184,0.92152,2.7786,0.41264,2.3382,3.1647,1.2647,1.1072,-3.2381,-1.2591,-0.83185,-1.6059,0.57993,-2.118,-0.54878,0.76078,-0.074174,0.30904,-0.28318,1.7597,-0.83301,-2.5778,-3.7242,0.58209,3.8249,1.4794,-2.242,-2.0715,0.82211,3.4849,-1.6172,-4.0141,-0.16727,-2.0282,7.0865,0.84765,-3.4945,1.2765,2.2956,0.47523,2.3143,-0.93912,0.030626,-1.4647,-0.42383,1.7943,-0.78176,-3.8079,0.45426,0.58407,-0.15175,2.4248,-3.8759,-0.19335,-3.5155,-1.3907,-1.2319,0.48635,1.6602,0.84975,4.6175,-0.81367,0.47439,1.6774,1.7393,4.3165,0.14046,1.4182,-1.8713,-1.4948,-1.0273,-1.264,2.5117,2.1463,-1.052,2.5754,-2.8909,2.0706,-0.56698,2.6587,2.5999,-4.9548,-2.0778,-0.67968,-0.62967,-2.5925,-1.5708,-2.252,-0.10327,1.2572,-2.7904,2.8963,3.1598,2.2888,3.2515,-0.4181,2.1012,2.7386,-0.1432,-0.35501,-1.7387,1.5427,1.7338,-2.4797,1.0412,1.8452,-0.01234,-1.6399,-1.29,-0.055819,3.7824,0.87042,3.8564,0.43643,0.13927,-2.9037,0.46992,0.76273,0.52487,-2.5186,2.5555,-1.1517,-1.5503,-2.034,-2.3307,-0.35606,0.2412,-5.179,-3.2397,-1.4073,0,0


### Pycaret Regression: 300 inputs and 1 numerical outputs

In [159]:
exp_regr = pycaret.regression.setup(df_last, target='target', session_id=123)

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Regression
3,Original data shape,"(3511, 301)"
4,Transformed data shape,"(3511, 301)"
5,Transformed train set shape,"(2457, 301)"
6,Transformed test set shape,"(1054, 301)"
7,Numeric features,300
8,Preprocess,True
9,Imputation type,simple


In [160]:
exp_regr.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:31:26
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


[]

In [100]:
exp_regr.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,02:34:55
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0330
en,Elastic Net,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0300
dummy,Dummy Regressor,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0300
llar,Lasso Least Angle Regression,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0280
br,Bayesian Ridge,0.1552,0.0351,0.1873,-0.0076,0.1314,0.5705,0.0320
omp,Orthogonal Matching Pursuit,0.1552,0.0352,0.1875,-0.0094,0.1315,0.5700,0.0290
lr,Linear Regression,0.1562,0.0357,0.1887,-0.0228,0.1324,0.5732,0.0310
lar,Least Angle Regression,0.1560,0.0357,0.1887,-0.0233,0.1324,0.5724,0.0290
ridge,Ridge Regression,0.1562,0.0357,0.1888,-0.0239,0.1325,0.5725,0.0300
ada,AdaBoost Regressor,0.1579,0.0360,0.1895,-0.0318,0.1339,0.6074,0.1460


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000, positive=False,
      precompute=False, random_state=123, selection='cyclic', tol=0.0001,
      warm_start=False)

### Pycaret Classification

In [90]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,target
periodical,-1.627200,-0.782380,1.666100,0.64415,5.802500,-3.86060,2.11450,2.246400,0.11389,-0.57345,...,-1.15170,-1.55030,-2.03400,-2.33070,-0.35606,0.241200,-5.179000,-3.23970,-1.40730,Contextual
intellect,-0.154690,1.676800,-0.351630,-0.57609,2.088700,2.61040,0.66495,2.229900,-4.99240,-1.65200,...,-1.79910,-1.50010,-3.83950,1.14140,0.38565,1.783600,-0.127660,-5.47600,5.23490,Latent
culture,-0.025891,-2.009300,0.494080,-0.33639,5.294000,-0.38846,3.17880,2.941100,-2.58270,-1.90800,...,-0.75585,-0.47891,-2.78880,-0.65771,-4.51980,1.242700,-1.250500,-7.25270,5.32900,Latent
emancipation,-3.593900,-0.710900,2.035000,2.18820,2.327500,0.66056,2.73950,0.424270,1.17990,-0.65433,...,-0.22063,-1.37600,2.01930,1.47420,-0.93661,0.339360,-4.583200,-1.69060,-0.82634,Contextual
chasm,2.750600,2.573200,1.016900,-2.14450,-1.060800,1.80230,3.07630,3.057700,-1.46700,1.42200,...,0.71314,3.44560,-1.67040,2.73970,1.79250,-1.171300,0.825910,1.09540,-0.33145,Latent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
prawn,-1.104300,-2.707300,-0.040561,0.26686,-0.001637,-2.23060,-0.22431,0.000931,-3.39360,0.79063,...,-2.56200,1.02140,-2.52040,-1.98950,1.48670,1.859700,-0.720330,-1.17220,4.63340,Manifest
tweezers,-1.051400,2.791100,1.661000,2.09520,-0.197260,-1.18820,-2.49820,0.762750,-3.10200,0.28440,...,-0.44270,1.51110,0.88290,-2.94660,0.53499,0.968740,-0.909550,-0.77088,-1.42830,Manifest
university,-1.409000,0.093909,0.172970,-0.25512,2.648800,2.68320,3.31220,3.572100,-0.04109,0.20994,...,-1.17040,-1.25060,-0.83331,0.25252,0.19663,1.934500,-0.546330,0.59882,0.67005,Perceptual
wasteful,-1.105000,-0.335100,-1.827500,-3.49660,1.209300,1.20280,0.13930,-1.561000,-0.51685,-0.91023,...,-0.92690,2.68170,-0.71914,-1.27960,0.42766,-1.082500,0.035378,-3.72800,1.17320,Latent


In [161]:
## Fazendo um experimento:
exp_class = pycaret.classification.setup(df_last, target='target', session_id=9088)

,Description,Value
0,Session id,9088
1,Target,target
2,Target type,Multiclass
3,Target mapping,"Contextual: 0, Latent: 1, Manifest: 2, Perceptual: 3"
4,Original data shape,"(3511, 301)"
5,Transformed data shape,"(3511, 301)"
6,Transformed train set shape,"(2457, 301)"
7,Transformed test set shape,"(1054, 301)"
8,Numeric features,300
9,Preprocess,True


In [162]:
exp_class.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:32:04
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.6789,0.0000,0.6789,0.6689,0.6675,0.5516,0.5552,0.0300
lda,Linear Discriminant Analysis,0.6777,0.8846,0.6777,0.6770,0.6758,0.5564,0.5573,0.0520
lightgbm,Light Gradient Boosting Machine,0.6712,0.8857,0.6712,0.6594,0.6541,0.5381,0.5441,2.1390
gbc,Gradient Boosting Classifier,0.6590,0.8830,0.6590,0.6475,0.6451,0.5227,0.5270,14.3410
lr,Logistic Regression,0.6443,0.8568,0.6443,0.6458,0.6440,0.5121,0.5128,0.5550
svm,SVM - Linear Kernel,0.6325,0.0000,0.6325,0.6294,0.6254,0.4924,0.4956,0.0690
rf,Random Forest Classifier,0.6305,0.8571,0.6305,0.6097,0.5810,0.4691,0.4888,0.4190
et,Extra Trees Classifier,0.6260,0.8647,0.6260,0.6127,0.5636,0.4588,0.4857,0.1000
knn,K Neighbors Classifier,0.6215,0.8314,0.6215,0.6134,0.6043,0.4684,0.4747,0.0360
qda,Quadratic Discriminant Analysis,0.6125,0.7806,0.6125,0.5969,0.4932,0.4298,0.4790,0.0460


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=9088, solver='auto',
                tol=0.0001)